In [1]:
import numpy as np

In [2]:
a = np.ones((2,1))
a[1] = 2
a

array([[1.],
       [2.]])

In [3]:
a2 = np.ones((3,1)) *2
a2

array([[2.],
       [2.],
       [2.]])

In [4]:
b = np.ones(4)
b[1] = 2
b[2] = 3
b[3] = 4
b

array([1., 2., 3., 4.])

In [5]:
c = a*b
d = a2*b
c

array([[1., 2., 3., 4.],
       [2., 4., 6., 8.]])

In [6]:
e = np.random.uniform((-2/np.sqrt(4)), (2/np.sqrt(4)), 4* 7)
e = np.reshape(e, (4, 7))
np.matmul(d, e)

array([[  8.13169596,  -5.65649335,  -4.24872545, -10.34095102,
         -2.60426131,  -4.00052107,   5.69724286],
       [  8.13169596,  -5.65649335,  -4.24872545, -10.34095102,
         -2.60426131,  -4.00052107,   5.69724286],
       [  8.13169596,  -5.65649335,  -4.24872545, -10.34095102,
         -2.60426131,  -4.00052107,   5.69724286]])

In [7]:
d

array([[2., 4., 6., 8.],
       [2., 4., 6., 8.],
       [2., 4., 6., 8.]])

Figure out how to merge

In [8]:
#aint it
np.outer(c,d)

array([[ 2.,  4.,  6.,  8.,  2.,  4.,  6.,  8.,  2.,  4.,  6.,  8.],
       [ 4.,  8., 12., 16.,  4.,  8., 12., 16.,  4.,  8., 12., 16.],
       [ 6., 12., 18., 24.,  6., 12., 18., 24.,  6., 12., 18., 24.],
       [ 8., 16., 24., 32.,  8., 16., 24., 32.,  8., 16., 24., 32.],
       [ 4.,  8., 12., 16.,  4.,  8., 12., 16.,  4.,  8., 12., 16.],
       [ 8., 16., 24., 32.,  8., 16., 24., 32.,  8., 16., 24., 32.],
       [12., 24., 36., 48., 12., 24., 36., 48., 12., 24., 36., 48.],
       [16., 32., 48., 64., 16., 32., 48., 64., 16., 32., 48., 64.]])

In [9]:
#I think this is right?
res = np.einsum("ik,jk->kij",c,d)
res

array([[[ 2.,  2.,  2.],
        [ 4.,  4.,  4.]],

       [[ 8.,  8.,  8.],
        [16., 16., 16.]],

       [[18., 18., 18.],
        [36., 36., 36.]],

       [[32., 32., 32.],
        [64., 64., 64.]]])

In [10]:
np.shape(res)

(4, 2, 3)

In [11]:
import torch
from siren import Sine #pip install siren-torch

In [12]:
class coordx_net(torch.nn.Module):
    def __init__(self):
        super(coordx_net, self).__init__()
        self.x_first = torch.nn.Linear(1,16)
        self.y_first = torch.nn.Linear(1,16)
        self.premerge_parallels = torch.nn.Sequential(
            Sine(),
            torch.nn.Linear(16,16),
            Sine(),
            torch.nn.Linear(16,16),
            Sine()
        )
        
        self.postmerge = torch.nn.Sequential(
            torch.nn.Linear(16,32),
            Sine(),
            torch.nn.Linear(32,3),
            torch.nn.Sigmoid()
        )
    
    def merge(self,x,y):
        return torch.einsum("ik,jk->ijk",x,y)

    def forward(self, x, y):
        x1 = self.x_first(x)
        y1 = self.y_first(y)
        x2 = self.premerge_parallels(x1)
        y2 = self.premerge_parallels(y1)
        merged = self.merge(x2,y2)
        print(merged.shape)
        res = self.postmerge(merged)
        print(res.shape)
        return res
    

In [14]:
net = coordx_net()
x = torch.ones((2,1)) * 1
print("x", x)
print("sine", Sine()(x))
y = torch.ones((3,1)) * 2
print( "y", y)
merged = net(x, y)
merged

x tensor([[1.],
        [1.]])
sine tensor([[0.8415],
        [0.8415]])
y tensor([[2.],
        [2.],
        [2.]])
torch.Size([2, 3, 16])
torch.Size([2, 3, 3])


tensor([[[0.5038, 0.4897, 0.4738],
         [0.5038, 0.4897, 0.4738],
         [0.5038, 0.4897, 0.4738]],

        [[0.5038, 0.4897, 0.4738],
         [0.5038, 0.4897, 0.4738],
         [0.5038, 0.4897, 0.4738]]], grad_fn=<SigmoidBackward0>)